# Table: active

**Motivation**: Proportion of active neurons. Device = ```cuda:1``` <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-vae/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-vae/figs')
tmp_dir = os.path.join(git_dir, 'jb-vae/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_PoissonVAE'))
from analysis.eval import sparse_score
from figures.fighelper import *
from vae.train_vae import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

## Device

In [2]:
device_idx = 1
device = f'cuda:{device_idx}'

from analysis.final import sort_fits, analyze_fits, add_nelbo_diff
from figures.table import table_entry_active
from analysis.stats import ttest

## Load dfs

In [3]:
df_main = pd.read_pickle(pjoin(tmp_dir, 'df_main.df'))

selected_models = [
    'poisson',
    'gaussian',
    'laplace',
]
df = df_main.loc[
    (df_main['method'] == 'mc') &
    (df_main['type'].isin(selected_models))
]

df.shape

(90, 17)

In [4]:
df.pivot_table(
    values='active',
    index='type',
    columns=['dataset', 'enc_type'],
)

dataset  CIFAR10-PATCHES               DOVES               MNIST          
enc_type            conv       lin      conv       lin      conv       lin
type                                                                      
gaussian        0.246094  0.105469  0.246094  0.217578  0.031250  0.027344
laplace         0.230469  0.192969  0.221875  0.187500  0.033984  0.027344
poisson         0.928125  0.999219  0.819141  0.983984  0.426172  0.536719

### ttest

In [5]:
df_agg, df_ttest = ttest(
    df=df,
    value='active',
    by=['dataset', 'enc_type', 'type'],
    test_conds=[
        ('poisson', m) for m in
        df['type'].unique()
        if m != 'poisson'],
    method='fdr_bh',
    alpha=0.01,
)

In [6]:
df_ttest

,dataset,enc_type,cond1,cond2,t,pvals,pvals_fdr_bh,reject
0,CIFAR10-PATCHES,conv,poisson,gaussian,61.982740,4.058027e-07,4.869632e-07,True
1,CIFAR10-PATCHES,conv,poisson,laplace,63.402735,3.706802e-07,4.869632e-07,True
2,CIFAR10-PATCHES,lin,poisson,gaussian,407.662481,2.172355e-10,1.303413e-09,True
3,CIFAR10-PATCHES,lin,poisson,laplace,842.624472,1.190175e-11,1.428210e-10,True
4,DOVES,conv,poisson,gaussian,58.147482,5.238076e-07,5.714265e-07,True
5,DOVES,conv,poisson,laplace,56.552108,5.853981e-07,5.853981e-07,True
6,DOVES,lin,poisson,gaussian,255.430643,1.409337e-09,2.818674e-09,True
7,DOVES,lin,poisson,laplace,300.634149,7.344563e-10,2.818674e-09,True
8,MNIST,conv,poisson,gaussian,141.568316,1.493284e-08,2.559916e-08,True
9,MNIST,conv,poisson,laplace,134.165143,1.851107e-08,2.776661e-08,True


In [7]:
df_ttest.pivot(index='cond2', columns=['dataset', 'enc_type'], values='reject')

dataset  CIFAR10-PATCHES       DOVES       MNIST      
enc_type            conv   lin  conv   lin  conv   lin
cond2                                                 
gaussian            True  True  True  True  True  True
laplace             True  True  True  True  True  True

In [8]:
for index, values in df_agg.items():
    print(index, len(values))
    break

('CIFAR10-PATCHES', 'conv', 'gaussian')
5

### ci

In [11]:
%%time

for ds in ['DOVES', 'CIFAR10-PATCHES', 'MNIST']:
    table = table_entry_active(ds, df_agg, fmt='0.3f')
    print(f"——— dataset: {ds} ———\n")
    print(table)
    print('\n\n\n')

——— dataset: DOVES ———

\begin{tabularx}{34mm}{CC}
    \entry{0.984}{.011} & \entry{0.819}{.041} \\ 
    \entry{0.188}{.000} & \entry{0.222}{.003} \\ 
    \entry{0.218}{.003} & \entry{0.246}{.000}
\end{tabularx}

——— dataset: CIFAR10-PATCHES ———

\begin{tabularx}{34mm}{CC}
    \entry{0.999}{.002} & \entry{0.928}{.045} \\ 
    \entry{0.193}{.003} & \entry{0.230}{.000} \\ 
    \entry{0.105}{.008} & \entry{0.246}{.000}
\end{tabularx}

——— dataset: MNIST ———

\begin{tabularx}{34mm}{CC}
    \entry{0.537}{.008} & \entry{0.426}{.011} \\ 
    \entry{0.027}{.000} & \entry{0.034}{.002} \\ 
    \entry{0.027}{.000} & \entry{0.031}{.000}
\end{tabularx}

CPU times: user 2.18 s, sys: 441 ms, total: 2.62 s
Wall time: 2.62 s
